In [3]:
import pandas as pd
import numpy as np
import math
# from data.get_data import run_query, install_backlog_sql, maint_backlog_sql, live_fleet_sql, new_sales_sla_sql, locs_in_implementation_sql

import functions.main_functions as mf
import functions.report_functions as rf

input_data = 'binder TEMPLATE Tech Hiring.xlsx'
install_res_dict, maint_res_dict, wo_tech_mnthly_rr_less_ss = mf.run_model(input_data)

### Visuals

In [4]:
import altair as alt
# alt.renderers.enable("jupyter")

# intall_df.index.name = 'Service Territory'
cap_perc_ma3 = rf.get_cap_perc_ma3(install_res_dict, wo_tech_mnthly_rr_less_ss)
cap_perc = rf.get_cap_perc(install_res_dict, wo_tech_mnthly_rr_less_ss)

chart_df = cap_perc_ma3.reset_index()

long_install_df = pd.melt(chart_df, id_vars='Service Territory', value_vars=cap_perc_ma3.columns, value_name='cap_perc', var_name='date')
long_install_df.set_index('Service Territory', inplace=True)
long_install_df = long_install_df.reset_index()

source = long_install_df

chart = alt.Chart(source).encode(alt.Color("Service Territory").legend(None)).properties(
    width=900,
    height=600
)
# .transform_filter(alt.datum.symbol)
line = chart.mark_line().encode(x='date:T', y='cap_perc:Q')
label = chart.encode(x='max(date):T', y=alt.Y('cap_perc:Q').aggregate(argmax='variable'), text='Service Territory')
circle = label.mark_circle()
line + circle

alt.LayerChart(...)

### Write to excel

In [6]:

writer = pd.ExcelWriter('output/tech_hiring_model_output.xlsx', engine='openpyxl') 

cap_perc_ma3.to_excel(writer, sheet_name='cap_perc_ma3')
cap_perc.to_excel(writer, sheet_name='cap_perc')

wo_tech_mnthly_rr_less_ss.to_excel(writer, sheet_name='monthly_run_rate')
install_res_dict['qtrly_tech_cap'].to_excel(writer, sheet_name='tech_count')

install_res_dict['install_dt_unconstrained_v2'].to_excel(writer, sheet_name='install_dt')
maint_res_dict['maint_dt_unconstrained_v2'].to_excel(writer, sheet_name='maint_dt')

install_res_dict['local_tech_supply'].to_excel(writer, sheet_name='local_tech_supply')
install_res_dict['travel_tech_supply'].to_excel(writer, sheet_name='travel_tech_supply')

maint_res_dict['local_tech_maint_supply'].to_excel(writer, sheet_name='local_tech_maint_supply')
maint_res_dict['travel_tech_maint_supply'].to_excel(writer, sheet_name='travel_tech_maint_supply')

install_res_dict['internal_tech_supply_df'].to_excel(writer, sheet_name='internal_tech_supply')
install_res_dict['external_tech_supply_df'].to_excel(writer, sheet_name='external_tech_supply')

maint_res_dict['internal_maint_tech_supply_df'].to_excel(writer, sheet_name='internal_maint_tech_supply')
maint_res_dict['external_maint_tech_supply_df'].to_excel(writer, sheet_name='external_maint_tech_supply')

maint_res_dict['nsa_external_maint_tech_supply_df'].to_excel(writer, sheet_name='nsa_external_maint_tech_supply')
maint_res_dict['dish_external_maint_tech_supply_df'].to_excel(writer, sheet_name='dish_external_maint_tech_supply')

install_res_dict['intall_df'].to_excel(writer, sheet_name='unmet_install_dt')
maint_res_dict['maint_df'].to_excel(writer, sheet_name='unmet_maint_df')

# roll_install_delta_dt_v_int_ext.to_excel(writer, sheet_name='roll_install_delta_dt_v_int_ext')


writer.close()